# Offspring Generation with ComfyUI - Version 4.0

Were getting very similar generated photos in earlier results despite randomizing parameters and changing the prompt to be less specific. Now testing blend ratios of source files (generated donor photos and mine).

The scenarios include:
- Random ratio (0.2-0.8)
- Donor dominant (80%)
- Balanced (50/50)
- Me dominant (80%)

Goal 🤞🏼: 85 donor profiles with 4 scenarios each (340 total generations)

- This one failed because it blended mine and the donor's photos, but not generating offspring

---

## 1. Setup and Imports

In [1]:
# Basic imports
import sys
import os
from pathlib import Path
import json
import csv
from datetime import datetime
import random
import logging
import shutil
import time
from typing import Dict, List, Tuple, Optional

In [2]:
# Server-related imports
import subprocess
import webbrowser
from time import sleep

In [3]:
# Configure logging
# Re-run this cell if logging needs to be restarted
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## 2. Configuration and Constants

In [5]:
# Base paths
COMFY_BASE = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI")

In [6]:
PATHS = {
    'output': COMFY_BASE / "output",
    'workflows': COMFY_BASE / "output/workflows",
    'logs': COMFY_BASE / "output/logs",
    'donors': Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files"),
    'my_photo': Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/cindy.jpg")
}

BASE_WORKFLOW_PATH = PATHS['workflows'] / "base_workflow_comfyui4.json"

In [7]:
# Blend scenarios and naming
BLEND_SCENARIOS = {
    'random': (0.2, 0.8),
    'donor_dominant': 0.8,
    'balanced': 0.5,
    'me_dominant': 0.8
}

OUTPUT_NAMING = {
    'random': 'rand{ratio}',
    'donor_dominant': 'donor80',
    'balanced': 'blend50',
    'me_dominant': 'me80'
}


In [8]:
# Prompt variations for more diverse results
PROMPT_VARIATIONS = {
    'age_range': (10, 14),  # months
    'expressions': [
        'candid', 'curious', 'happy', 'neutral', 
        'gentle smile', 'attentive', 'peaceful'
    ],
    'lighting_conditions': [
        'natural daylight', 'soft studio lighting', 
        'gentle window light', 'professional portrait lighting'
    ]
}

# Model parameters
MODEL_PARAMS = {
    'denoise_strength': (0.4, 0.7),
    'cfg_scale': (4, 7),
    'ip_adapter_strength': (0.6, 0.8),
    'controlnet_strength': (0.6, 0.8)
}


In [ ]:
base_workflow_json = {
    "last_node_id": 8,
    "last_link_id": 12,
    "nodes": [
        {
            "id": 1,
            "type": "LoadImage",
            "pos": [
                50,
                200
            ],
            "size": [
                315,
                98
            ],
            "flags": {},
            "order": 0,
            "mode": 0,
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [3],
                    "slot_index": 0
                }
            ],
            "properties": {
                "Node name for S&R": "LoadImage"
            }
        },
        {
            "id": 2,
            "type": "LoadImage",
            "pos": [
                50,
                400
            ],
            "size": [
                315,
                98
            ],
            "flags": {},
            "order": 1,
            "mode": 0,
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [4],
                    "slot_index": 0
                }
            ],
            "properties": {
                "Node name for S&R": "LoadImage"
            }
        },
        {
            "id": 3,
            "type": "VAEEncode",
            "pos": [
                400,
                200
            ],
            "size": [
                210,
                46
            ],
            "flags": {},
            "order": 2,
            "mode": 0,
            "inputs": [
                {
                    "name": "pixels",
                    "type": "IMAGE",
                    "link": 3
                }
            ],
            "outputs": [
                {
                    "name": "LATENT",
                    "type": "LATENT",
                    "links": [5],
                    "slot_index": 0
                }
            ]
        },
        {
            "id": 4,
            "type": "VAEEncode",
            "pos": [
                400,
                400
            ],
            "size": [
                210,
                46
            ],
            "flags": {},
            "order": 3,
            "mode": 0,
            "inputs": [
                {
                    "name": "pixels",
                    "type": "IMAGE",
                    "link": 4
                }
            ],
            "outputs": [
                {
                    "name": "LATENT",
                    "type": "LATENT",
                    "links": [6],
                    "slot_index": 0
                }
            ]
        },
        {
            "id": 5,
            "type": "LatentBlend",
            "pos": [
                650,
                300
            ],
            "size": [
                210,
                82
            ],
            "flags": {},
            "order": 4,
            "mode": 0,
            "inputs": [
                {
                    "name": "samples1",
                    "type": "LATENT",
                    "link": 5
                },
                {
                    "name": "samples2",
                    "type": "LATENT",
                    "link": 6
                }
            ],
            "outputs": [
                {
                    "name": "LATENT",
                    "type": "LATENT",
                    "links": [7],
                    "slot_index": 0
                }
            ],
            "properties": {
                "Node name for S&R": "LatentBlend"
            },
            "widgets_values": [
                0.5
            ]
        },
        {
            "id": 6,
            "type": "VAEDecode",
            "pos": [
                900,
                300
            ],
            "size": [
                210,
                46
            ],
            "flags": {},
            "order": 5,
            "mode": 0,
            "inputs": [
                {
                    "name": "samples",
                    "type": "LATENT",
                    "link": 7
                }
            ],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [8],
                    "slot_index": 0
                }
            ]
        },
        {
            "id": 7,
            "type": "SaveImage",
            "pos": [
                1150,
                300
            ],
            "size": [
                210,
                270
            ],
            "flags": {},
            "order": 6,
            "mode": 0,
            "inputs": [
                {
                    "name": "images",
                    "type": "IMAGE",
                    "link": 8
                }
            ],
            "properties": {},
            "widgets_values": [
                "ComfyUI"
            ]
        }
    ],
    "links": [
        [3, 1, 0, 3, 0, "IMAGE"],
        [4, 2, 0, 4, 0, "IMAGE"],
        [5, 3, 0, 5, 0, "LATENT"],
        [6, 4, 0, 5, 1, "LATENT"],
        [7, 5, 0, 6, 0, "LATENT"],
        [8, 6, 0, 7, 0, "IMAGE"]
    ],
    "groups": [],
    "config": {},
    "extra": {},
    "version": 0.4
}

## 3. Utility Functions

In [10]:
def setup_directories() -> None:
    """Create necessary directories and verify paths."""
    try:
        for name, path in PATHS.items():
            if name != 'my_photo':
                path.mkdir(parents=True, exist_ok=True)
                logging.info(f"Created/verified directory: {path}")
        
        # Verify my_photo exists
        if not PATHS['my_photo'].exists():
            raise FileNotFoundError(f"My photo not found at: {PATHS['my_photo']}")
        
        # Create base workflow if it doesn't exist
        if not BASE_WORKFLOW_PATH.exists():
            with open(BASE_WORKFLOW_PATH, 'w') as f:
                json.dump(base_workflow_json, f, indent=2)
            logging.info(f"Created base workflow at: {BASE_WORKFLOW_PATH}")
            
    except Exception as e:
        logging.error(f"Error in directory setup: {str(e)}")
        raise

In [11]:
def generate_random_prompt() -> str:
    """Generate a randomized prompt with variations.
    Each prompt will have random age, expression, and lighting."""
    age = random.randint(*PROMPT_VARIATIONS['age_range'])
    expression = random.choice(PROMPT_VARIATIONS['expressions'])
    lighting = random.choice(PROMPT_VARIATIONS['lighting_conditions'])
    
    return (
        f"portrait of a {age} month old baby with distinctive facial features, "
        f"{expression} expression, detailed facial features, {lighting}, "
        f"high quality photograph, photorealistic"
    )

def get_random_parameters() -> Dict:
    """Generate random parameters within defined ranges.
    Creates a complete set of parameters for ComfyUI workflow."""
    return {
        'denoise_strength': random.uniform(*MODEL_PARAMS['denoise_strength']),
        'cfg_scale': random.uniform(*MODEL_PARAMS['cfg_scale']),
        'ip_adapter_strength': random.uniform(*MODEL_PARAMS['ip_adapter_strength']),
        'controlnet_strength': random.uniform(*MODEL_PARAMS['controlnet_strength']),
        'seed': random.randint(1, 2**32 - 1)
    }

def get_blend_ratio(scenario: str) -> float:
    """Get blend ratio based on scenario.
    For random scenario, generates a ratio between 0.2-0.8.
    For other scenarios, uses predefined values."""
    if scenario == 'random':
        return round(random.uniform(*BLEND_SCENARIOS[scenario]), 2)
    return BLEND_SCENARIOS[scenario]

## 4. Logging Functions

In [12]:
class GenerationLogger:
    """Handles logging of generation parameters and results.
    Creates two types of logs:
    1. CSV log for quick overview
    2. Detailed JSON logs for each generation"""
    
    def __init__(self, log_dir: Path):
        self.log_dir = log_dir
        self.csv_path = log_dir / 'generation_log.csv'
        self.json_dir = log_dir / 'detailed_logs'
        self.json_dir.mkdir(parents=True, exist_ok=True)
        
        # Initialize CSV if it doesn't exist
        if not self.csv_path.exists():
            self._initialize_csv()
    
    def _initialize_csv(self):
        headers = [
            'timestamp', 'donor_id', 'scenario', 'blend_ratio',
            'seed', 'denoise_strength', 'cfg_scale',
            'ip_adapter_strength', 'controlnet_strength',
            'output_filename', 'prompt'
        ]
        with open(self.csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(headers)
    
    def log_generation(self, 
                      donor_id: str,
                      scenario: str,
                      parameters: Dict,
                      output_filename: str,
                      prompt: str):
        """Log a single generation to both CSV and JSON.
        Creates both a CSV entry and a detailed JSON log file."""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # CSV logging
        csv_row = [
            timestamp, donor_id, scenario, parameters['blend_ratio'],
            parameters['seed'], parameters['denoise_strength'],
            parameters['cfg_scale'], parameters['ip_adapter_strength'],
            parameters['controlnet_strength'], output_filename, prompt
        ]
        
        with open(self.csv_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(csv_row)
        
        # Detailed JSON logging
        json_log = {
            'timestamp': timestamp,
            'donor_id': donor_id,
            'scenario': scenario,
            'parameters': parameters,
            'output_filename': output_filename,
            'prompt': prompt
        }
        
        json_path = self.json_dir / f"{donor_id}_{timestamp}.json"
        with open(json_path, 'w') as f:
            json.dump(json_log, f, indent=2)



In [13]:
class WorkflowTracker:
    """Tracks the progress of workflow processing.
    Keeps track of which workflows have been processed and any notes/errors."""
    
    def __init__(self, tracker_dir: Path):
        self.tracker_dir = tracker_dir
        self.tracker_file = tracker_dir / 'processed_workflows.csv'
        self._initialize_tracker()
        
    def _initialize_tracker(self):
        """Initialize tracker file if it doesn't exist."""
        if not self.tracker_file.exists():
            with open(self.tracker_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['workflow_file', 'donor_id', 'scenario', 'timestamp', 'processed', 'notes'])
    
    def add_workflow(self, workflow_file: str, donor_id: str, scenario: str):
        """Add new workflow to tracking."""
        with open(self.tracker_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                workflow_file,
                donor_id,
                scenario,
                datetime.now().strftime('%Y%m%d_%H%M%S'),
                'No',
                ''
            ])
    
    def mark_processed(self, workflow_file: str, notes: str = ''):
        """Mark workflow as processed and add any notes about the result."""
        # Read existing data
        with open(self.tracker_file, 'r', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
            header = data[0]
            rows = data[1:]
        
        # Update processed status
        for row in rows:
            if row[0] == workflow_file:
                row[4] = 'Yes'
                row[5] = notes
        
        # Write back
        with open(self.tracker_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(rows)
    
    def get_unprocessed(self) -> List[str]:
        """Get list of unprocessed workflows."""
        unprocessed = []
        with open(self.tracker_file, 'r', newline='') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            for row in reader:
                if row[4] == 'No':
                    unprocessed.append(row[0])
        return unprocessed


## 5. Workflow Management

In [14]:
class WorkflowManager:
    def __init__(self, base_workflow_path: Path):
        self.base_workflow_path = base_workflow_path
        self.load_base_workflow()
    
    def load_base_workflow(self):
        """Load the base workflow JSON."""
        try:
            with open(self.base_workflow_path, 'r') as f:
                self.base_workflow = json.load(f)
            logging.info("Base workflow loaded successfully")
        except Exception as e:
            logging.error(f"Error loading base workflow: {str(e)}")
            raise
    
    def create_variation_workflow(self,
                               my_photo_path: Path,
                               donor_photo_path: Path,
                               parameters: Dict,
                               output_filename: str) -> Dict:
        """Create a workflow variation with specified parameters."""
        workflow = self.base_workflow.copy()
        
        # Update LoadImage nodes
        for node in workflow['nodes']:
            if node['type'] == 'LoadImage':
                if node['id'] == 1:  # Your photo
                    node['widgets_values'] = [str(my_photo_path)]
                elif node['id'] == 2:  # Donor photo
                    node['widgets_values'] = [str(donor_photo_path)]
            
            # Update LatentBlend node parameters
            elif node['type'] == 'LatentBlend':
                node['widgets_values'] = [parameters['blend_ratio']]
            
            # Update SaveImage node
            elif node['type'] == 'SaveImage':
                node['widgets_values'] = [output_filename]
        
        return workflow
    
    def verify_workflow_structure(self):
        """Verify that workflow has all required nodes."""
        required_nodes = {
            'LoadImage': 2,  # Need two of these
            'VAEEncode': 2,
            'LatentBlend': 1,
            'VAEDecode': 1,
            'SaveImage': 1
        }
        
        node_counts = {}
        for node in self.base_workflow['nodes']:
            node_type = node['type']
            node_counts[node_type] = node_counts.get(node_type, 0) + 1
        
        for node_type, required_count in required_nodes.items():
            actual_count = node_counts.get(node_type, 0)
            if actual_count < required_count:
                raise ValueError(
                    f"Workflow missing required nodes. Need {required_count} {node_type}, "
                    f"found {actual_count}"
                )

## 6. Main Processing

In [56]:
# Create base workflow
def create_base_workflow():
    """Create the base workflow JSON file."""
    base_workflow = {
    "last_node_id": 9,
    "last_link_id": 0,
    "nodes": [
        {
            "id": 1,
            "type": "LoadImage",
            "pos": [
                50,
                200
            ],
            "size": [
                315,
                102
            ],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [
                        5
                    ],
                    "slot_index": 0
                }
            ]
        },
        {
            "id": 2,
            "type": "LoadImage",
            "pos": [
                50,
                400
            ],
            "size": [
                315,
                102
            ],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [
                        6
                    ],
                    "slot_index": 0
                }
            ]
        },
        # ... rest of your workflow nodes
    ],
    "links": [
        [
            5,
            1,
            0,
            5,
            0,
            "IMAGE"
        ],
        [
            6,
            2,
            0,
            5,
            1,
            "IMAGE"
        ],
        [
            7,
            3,
            0,
            5,
            0,
            "MODEL"
        ],
        [
            8,
            3,
            1,
            4,
            0,
            "CLIP"
        ]
    ],
    "config": {},
    "extra": {
        "ds": {
            "scale": 1.5,
            "offset": [
                -1350,
                -211
            ]
        }
    }
}

    # Create workflows directory if it doesn't exist
    workflow_dir = PATHS['workflows']
    workflow_dir.mkdir(parents=True, exist_ok=True)
    
    # Save the base workflow
    workflow_path = workflow_dir / "base_workflow.json"
    with open(workflow_path, 'w') as f:
        json.dump(base_workflow, f, indent=2)
    
    logging.info(f"Created base workflow at: {workflow_path}")
    return workflow_path

In [57]:
def process_donor_batch():
    """Process a batch of donors with different scenarios."""
    try:
        # Initialize components
        setup_directories()
        logger = GenerationLogger(PATHS['logs'])
        workflow_manager = WorkflowManager(PATHS['workflows'] / "base_workflow_comfyui4.json")
        tracker = WorkflowTracker(PATHS['logs'])  # Initialize tracker here
        
        # Get donor files
        donor_files = sorted(list(PATHS['donors'].glob("*.jpeg")))
        logging.info(f"Found {len(donor_files)} donor files to process")
        
        # Process each donor
        for donor_idx, donor_path in enumerate(donor_files, 1):
            donor_id = donor_path.stem
            logging.info(f"\nProcessing donor {donor_idx}/{len(donor_files)}: {donor_id}")
            
            # Process each scenario
            for scenario in BLEND_SCENARIOS.keys():
                try:
                    # Get parameters
                    parameters = get_random_parameters()
                    parameters['blend_ratio'] = get_blend_ratio(scenario)
                    prompt = generate_random_prompt()
                    
                    # Generate output filename
                    suffix = OUTPUT_NAMING[scenario]
                    if scenario == 'random':
                        suffix = suffix.format(ratio=int(parameters['blend_ratio'] * 100))
                    output_filename = f"{donor_id}_{suffix}.png"
                    
                    # Create and save workflow
                    workflow = workflow_manager.create_variation_workflow(
                        PATHS['my_photo'],
                        donor_path,
                        parameters,
                        output_filename
                    )
                    
                    # Save workflow and add to tracker
                    workflow_file = f"{donor_id}_{suffix}.json"
                    workflow_path = PATHS['workflows'] / workflow_file
                    
                    with open(workflow_path, 'w') as f:
                        json.dump(workflow, f, indent=2)
                    
                    # Add to tracker
                    tracker.add_workflow(workflow_file, donor_id, scenario)
                    
                    # Log generation
                    logger.log_generation(
                        donor_id=donor_id,
                        scenario=scenario,
                        parameters=parameters,
                        output_filename=output_filename,
                        prompt=prompt
                    )
                    
                    logging.info(f"✓ Created workflow for {scenario} scenario: {output_filename}")
                    
                except Exception as e:
                    logging.error(f"Error processing {donor_id} - {scenario}: {str(e)}")
                    continue
            
            # Periodic save point
            if donor_idx % 10 == 0:
                logging.info(f"Completed {donor_idx}/{len(donor_files)} donors")
                
            time.sleep(1)  # Brief pause between donors
            
    except Exception as e:
        logging.error(f"Batch processing error: {str(e)}")
        raise
    
    logging.info("Batch processing completed!")

## 7. Server Management and Execution

In [58]:
def start_comfyui_server():
    """Start ComfyUI server and open browser interface.
    IMPORTANT: Keep this notebook running while using ComfyUI"""
    try:
        server_process = subprocess.Popen(
            ['python', str(COMFY_BASE / 'main.py'), '--listen', '0.0.0.0', '--port', '8188'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )

        # Wait for server to start
        sleep(5)

        # Open the UI in default browser
        webbrowser.open('http://localhost:8188')
        
        print("ComfyUI server started! The interface should open in your browser.")
        print("Keep this notebook running while you use ComfyUI.")
        
        return server_process
    except Exception as e:
        logging.error(f"Failed to start ComfyUI server: {str(e)}")
        raise

def check_progress():
    """Check which workflows still need processing.
    Run this function to see remaining workflows."""
    tracker = WorkflowTracker(PATHS['logs'])
    unprocessed = tracker.get_unprocessed()
    
    print(f"Unprocessed workflows: {len(unprocessed)}")
    for workflow in unprocessed:
        print(f"- {workflow}")
        
    return unprocessed

# Expected output: Server startup messages
# Common errors:
# - Port already in use: ComfyUI already running
# - Permission denied: Check Python permissions

### Execution Steps

In [37]:
# Step 1: Start ComfyUI server
server = start_comfyui_server()


ComfyUI server started! The interface should open in your browser.
Keep this notebook running while you use ComfyUI.


In [59]:
# Step 2: Create base workflow
base_workflow_path = create_base_workflow()

2024-11-22 20:18:53,561 - INFO - Created base workflow at: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/base_workflow.json


In [60]:
# Step 3: Generate all workflows
process_donor_batch()


2024-11-22 20:18:56,220 - INFO - Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output
2024-11-22 20:18:56,220 - INFO - Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows
2024-11-22 20:18:56,221 - INFO - Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/logs
2024-11-22 20:18:56,222 - INFO - Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files
2024-11-22 20:18:56,223 - INFO - Base workflow loaded successfully
2024-11-22 20:18:56,224 - INFO - Found 84 donor files to process
2024-11-22 20:18:56,224 - INFO - 
Processing donor 1/84: aug_cla_2_gem_wo
2024-11-22 20:18:56,227 - INFO - ✓ Created workflow for random scenario: aug_cla_2_gem_wo_rand36.png
2024-11-22 20:18:56,228 - INFO - ✓ Created workflow for donor_dominant scenario:

In [ ]:
# Step 4: Check progress
unprocessed = check_progress()

# Expected output: List of unprocessed workflows
# Use this to track progress

In [ ]:
# Helper function to mark workflows as processed
def mark_workflow_complete(workflow_file: str, notes: str = "Generated successfully"):
    """Mark a workflow as completed after running it in ComfyUI.
    Usage: mark_workflow_complete("donor_id_scenario.json")"""
    tracker = WorkflowTracker(PATHS['logs'])
    tracker.mark_processed(workflow_file, notes)
    print(f"Marked {workflow_file} as processed")

# Example usage:
# mark_workflow_complete("aug_cla_4_gem_wo_donor80.json", "Generated successfully")